In [28]:
%matplotlib inline

In [29]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

# Загружаем данные

In [9]:
macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

macro_cols_DB = [ "oil_urals", "usdrub", "eurrub" ,  "brent" , "gdp_annual" , "mortgage_value", 
"mortgage_rate", "grp", "income_per_cap", "salary", "construction_value",  "rent_price_2room_eco"
, "apartment_build" ,  "apartment_fund_sqm"]

df_train = pd.read_csv("./train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("./test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("./macro.csv", parse_dates=['timestamp'], usecols=['timestamp'] + macro_cols_DB)

df_train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


# Здесь добавляем новые признаки

In [49]:
# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
df_all = pd.merge_ordered(df_all, df_macro, on='timestamp', how='left')
print(df_all.shape)

# Add month-year - не понял, надо ли именно так делать
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count - не понял, надо ли именно так делать
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering - это фигня имхо, убрал
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

df_all[ 'elevator' ] = df_all[ 'max_floor' ] > 5
df_all[ 'is_first_floor' ] = df_all[ 'floor' ] == 1
df_all[ 'is_last_floor' ] = df_all[ 'floor' ] == df_all[ 'max_floor' ]
df_all[ 'age' ] = df_all.timestamp.dt.year - df_all.build_year

# Remove timestamp column (may overfit the model in train)
df_all.drop( ['timestamp'], axis=1, inplace=True )

(38133, 305)


In [50]:
# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_all = pd.concat([df_numeric, df_obj], axis=1)

# Сохраняем данные

In [51]:
df_train = df_all[ :num_train ].copy()
df_train.drop( ['id'], axis = 1, inplace = True )
df_test = df_all[ num_train: ].copy()
df_test.drop( ['price_doc'], axis = 1, inplace = True )

In [52]:
df_train.to_csv( './train_preprocessed.csv', index = False )
df_test.to_csv( './test_preprocessed.csv', index = False )